```{contents}
```
## **Cost Tracking in LangGraph**

**Cost Tracking** in LangGraph is the systematic measurement, attribution, and control of **monetary cost, token usage, and resource consumption** across complex LLM workflows.
It is a **first-class production concern** because LangGraph systems often involve **multiple models, agents, tools, loops, retries, and long-running executions**.

---

### **1. Why Cost Tracking Is Necessary**

In simple chains, cost ≈ one LLM call.
In LangGraph, cost accumulates across:

| Source            | Examples                    |
| ----------------- | --------------------------- |
| Multiple nodes    | Planner, executor, verifier |
| Cyclic loops      | Reflection, retries         |
| Parallel agents   | Multi-agent debate          |
| Tool calls        | Search APIs, databases      |
| Retries & backoff | Error recovery              |
| Human-in-the-loop | Long sessions               |

Without cost tracking, **real spend becomes unpredictable and dangerous in production**.

---

### **2. Cost Model in LangGraph**

Each graph execution produces **structured cost signals**:

| Layer   | What is Measured                 |
| ------- | -------------------------------- |
| LLM     | prompt tokens, completion tokens |
| Model   | per-token pricing                |
| Tool    | API fees                         |
| Runtime | execution time, memory           |
| Agents  | per-agent cost                   |
| Session | cumulative cost                  |
| Tenant  | per-user / per-organization cost |

Total cost =
**Σ (LLM tokens × price) + tool usage + infrastructure overhead**

---

### **3. Where Cost Is Collected**

LangGraph integrates with **LangChain callbacks**.

```
Graph Execution
   └─ Node
       └─ LLM / Tool
           └─ Callback → Cost Recorder
```

This allows **fine-grained cost attribution per node, per agent, per run**.

---

### **4. Minimal Cost Tracking Example**

```python
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    result = graph.invoke(input_state)

print("Total tokens:", cb.total_tokens)
print("Prompt tokens:", cb.prompt_tokens)
print("Completion tokens:", cb.completion_tokens)
print("Total cost ($):", cb.total_cost)
```

This captures **full-graph cost**, including loops and subgraphs.

---

### **5. Node-Level Cost Attribution**

```python
def llm_node(state):
    with get_openai_callback() as cb:
        response = llm.invoke(state["input"])
    return {
        "output": response,
        "node_cost": cb.total_cost
    }
```

You can accumulate this inside state:

```python
state["total_cost"] += state["node_cost"]
```

---

### **6. Agent-Level Cost Tracking**

Each agent maintains its own cost ledger:

| Agent     | Cost       |
| --------- | ---------- |
| Planner   | $0.021     |
| Executor  | $0.114     |
| Verifier  | $0.033     |
| **Total** | **$0.168** |

This enables:

* Budget enforcement per agent
* Optimization of expensive agents
* Dynamic model routing

---

### **7. Budget Enforcement & Circuit Breakers**

```python
def cost_guard(state):
    if state["total_cost"] > 1.00:
        return END
    return "continue"
```

Use as conditional edge:

```python
builder.add_conditional_edges("router", cost_guard, {
    "continue": "next_node",
    END: END
})
```

---

### **8. Cost-Aware Model Routing**

```python
def choose_model(state):
    if state["total_cost"] < 0.10:
        return cheap_model
    return premium_model
```

---

### **9. Cost Persistence & Analytics**

Store cost per:

* Graph run
* User session
* Tenant
* Time window

Use this for:

| Purpose      | Benefit                  |
| ------------ | ------------------------ |
| Billing      | Chargeback               |
| Monitoring   | Cost anomaly detection   |
| Optimization | Identify expensive paths |
| Governance   | Enforce policies         |

---

### **10. Production Best Practices**

| Principle           | Implementation         |
| ------------------- | ---------------------- |
| Always track        | Every graph run        |
| Cap budgets         | Hard limits            |
| Log costs           | Immutable logs         |
| Alert               | On cost spikes         |
| Optimize loops      | Prevent runaway cycles |
| Prefer cheap models | For early stages       |
| Batch calls         | Reduce overhead        |

---

### **11. Mental Model**

Think of LangGraph cost tracking as **distributed accounting**:

> Every node spends.
> Every agent accumulates.
> The graph enforces discipline.


### Demonstration

In [4]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_classic.callbacks import get_openai_callback

# -------------------- State --------------------

class State(TypedDict):
    input: str
    output: str
    total_cost: float
    steps: int

# -------------------- LLM --------------------

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# -------------------- Nodes --------------------

def llm_node(state: State):
    with get_openai_callback() as cb:
        response = llm.invoke(state["input"])
    return {
        "output": response.content,
        "total_cost": state["total_cost"] + cb.total_cost,
        "steps": state["steps"] + 1
    }

def cost_guard(state: State):
    if state["total_cost"] >= 0.01:     # hard budget
        return END
    if state["steps"] >= 5:             # hard step limit
        return END
    return "llm"

# -------------------- Graph --------------------

builder = StateGraph(State)
builder.add_node("llm", llm_node)
builder.set_entry_point("llm")

builder.add_conditional_edges("llm", cost_guard, {
    "llm": "llm",
    END: END
})

graph = builder.compile()

# -------------------- Run --------------------

result = graph.invoke(
    {
        "input": "Explain LangGraph in one sentence.",
        "output": "",
        "total_cost": 0.0,
        "steps": 0
    },
    config={"recursion_limit": 20}
)

print("Final Output:", result["output"])
print("Steps:", result["steps"])
print("Total Cost ($):", result["total_cost"])


Final Output: LangGraph is a tool that visualizes the relationships between languages based on their linguistic similarities.
Steps: 5
Total Cost ($): 0.0001785
